In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install langchain-gigachat langchain-community sentence-transformers langchain-huggingface chromadb langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.5 MB/s eta 0:00:00

## Базовый пример работы с API

In [ ]:
with open('/content/drive/MyDrive/ML_training_data/gigachat_api_key.txt', 'r') as f:
  api_key = f.read()

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_gigachat.chat_models import GigaChat

giga = GigaChat(
    credentials=api_key.strip(),
    verify_ssl_certs=False,
    scope="GIGACHAT_API_PERS"
)

In [ ]:
messages = [
    SystemMessage(
        content="Ты специалист по машинному обучению и цифровым гуманитарным наукам. Ты помогаешь пользователям, отвечая на их вопросы."
    )
]

while(True):
    user_input = input("Пользователь: ")
    if user_input == "пока":
      break
    messages.append(HumanMessage(content=user_input))
    res = giga.invoke(messages)
    messages.append(res)
    print("GigaChat: ", res.content)

Пользователь: привет! Расскажи, что такое RAG
GigaChat:  Привет! RAG — это сокращение от Red-Amber-Green (красный-желтый-зеленый). Это цветовая индикация, которая используется для отображения текущего состояния или прогресса чего-либо. Обычно красный цвет означает проблему или отставание, желтый — предупреждение или необходимость внимания, а зеленый — нормальное состояние или успешное завершение.

Применение RAG можно встретить в различных областях:
1. **Мониторинг процессов**: В системах мониторинга RAG используется для визуализации статуса серверов, приложений или других системных компонентов. Например, если сервер работает нормально, его статус будет зеленым; если есть проблемы, он станет красным.
2. **Проектный менеджмент**: В проектах RAG может использоваться для отслеживания прогресса задач или этапов проекта. Задачи могут быть помечены как "зеленые" (выполненные), "желтые" (в процессе выполнения) или "красные" (задержка или проблема).
3. **Анализ данных**: В аналитике данных RAG

## Retrieval Augmented Generation (RAG)

In [ ]:
question = "Что главный герой рассказа \"Скучная история\" Чехова думает про сад в университете, в котором работает?"

In [ ]:
messages = [
    SystemMessage(
        content="Ты - специалист по русской литературе 19-20 веков."
    ),
    HumanMessage(content=question),
]

print(giga.invoke(messages).content)

Главный герой рассказа "Скучная история" Чехова, известный профессор медицины, считает университетский сад совершенно ненужным и лишним украшением университета. Он говорит: "В нашем университете есть сад... Насаженный покойным ректором для умственного отдыха студентов, он давно уже не служит этой цели. Теперь это место для гулянья молодёжи..." (цитата из полного текста рассказа).


In [ ]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)

loader = TextLoader("/content/drive/MyDrive/ML_training_data/скучная история.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, # влияет на качество ответов!
    chunk_overlap=200,
)
documents = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

Total documents: 85


In [ ]:
documents[:5]

[Document(metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='\ufeffСкучная история\nАнтон Павлович Чехов\n\n\n\n\n\nСкучная история\n\n\n\nИз записок старого человека\n\n\nI'),
 Document(metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='Есть в России заслуженный профессор Николай Степанович такой-то, тайный советник и кавалер; у\xa0него так много русских и иностранных орденов, что когда ему приходится надевать их, то студенты величают его иконостасом. Знакомство у него самое аристократическое; по крайней мере, за последние двадцать пять – тридцать лет в России нет и не было такого знаменитого ученого, с которым он не был бы коротко знаком. Теперь дружить ему не с кем, но если говорить о прошлом, то длинный список его славных'),
 Document(metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='пять – тридцать лет в России нет и не было такого знаменито

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

model_name = "sentence-transformers/distiluse-base-multilingual-cased-v2"
model_kwargs = {'device': 'cpu'} # or cuda
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

2_Dense%2Fconfig.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

In [ ]:
from chromadb.config import Settings
from langchain_chroma import Chroma

In [ ]:
db = Chroma.from_documents(
    documents,
    hf,
    client_settings=Settings(anonymized_telemetry=False),
)

In [ ]:
docs = db.similarity_search(question, k=4)
len(docs)

4

In [ ]:
docs

[Document(id='3ca04f1d-7e51-45d9-b1e7-efa1a0b68214', metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='\ufeffСкучная история\nАнтон Павлович Чехов\n\n\n\n\n\nСкучная история\n\n\n\nИз записок старого человека\n\n\nI'),
 Document(id='16b68d42-9c8d-42f7-bbb6-c9febb520859', metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='наш сад. С тех пор как я был студентом, он, кажется, не стал ни лучше, ни хуже. Я его не люблю. Было бы гораздо умнее, если бы вместо чахоточных лип, желтой акации и редкой стриженой сирени росли тут высокие сосны и хорошие дубы. Студент, настроение которого в большинстве создается обстановкой, на каждом шагу, там, где он учится, должен видеть перед собою только высокое, сильное и изящное… Храни его бог от тощих деревьев, разбитых окон, серых стен и дверей, сбитых рваной клеенкой.'),
 Document(id='b1e082e9-f28f-4752-aff5-ba4673e464a1', metadata={'source': '/content/drive/MyDr

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "Ты должен ответить на вопрос пользователя с использованием данных из рассказа.\n"
    "Отвечаай коротко, не более 2-3 предложений.\n"
    "Вот контекст для ответа:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

retriever = db.as_retriever()

question_answer_chain = create_stuff_documents_chain(giga, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
rag_chain.invoke({"input": question})

{'input': 'Что главный герой рассказа "Скучная история" Чехова думает про сад в университете, в котором работает?',
 'context': [Document(id='3ca04f1d-7e51-45d9-b1e7-efa1a0b68214', metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='\ufeffСкучная история\nАнтон Павлович Чехов\n\n\n\n\n\nСкучная история\n\n\n\nИз записок старого человека\n\n\nI'),
  Document(id='16b68d42-9c8d-42f7-bbb6-c9febb520859', metadata={'source': '/content/drive/MyDrive/ML_training_data/скучная история.txt'}, page_content='наш сад. С тех пор как я был студентом, он, кажется, не стал ни лучше, ни хуже. Я его не люблю. Было бы гораздо умнее, если бы вместо чахоточных лип, желтой акации и редкой стриженой сирени росли тут высокие сосны и хорошие дубы. Студент, настроение которого в большинстве создается обстановкой, на каждом шагу, там, где он учится, должен видеть перед собою только высокое, сильное и изящное… Храни его бог от тощих деревьев, разбитых окон, серых стен и

In [ ]:
rag = rag_chain.invoke({"input": question})["answer"]
print(f"RAG: {rag}")

RAG: Главный герой считает, что сад университета неухожен и не соответствует высоким стандартам, необходимым для вдохновения студентов. Вместо чахоточных лип и желтой акации, он мечтает о высоких соснах и хороших дубах.


## Структурированная выдача

In [ ]:
from typing import Optional, Dict
from pydantic import BaseModel, Field

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
import pandas as pd

In [ ]:
reviews = pd.read_csv('/content/drive/MyDrive/ML_training_data/singapore_airlines_reviews.csv')

In [ ]:
reviews

,published_date,published_platform,rating,type,text,title,helpful_votes
0,2024-03-12T14:41:14-04:00,Desktop,3,review,We used this airline to go from Singapore to L...,Ok,0
1,2024-03-11T19:39:13-04:00,Desktop,5,review,The service on Singapore Airlines Suites Class...,The service in Suites Class makes one feel lik...,0
2,2024-03-11T12:20:23-04:00,Desktop,1,review,"Booked, paid and received email confirmation f...",Don’t give them your money,0
3,2024-03-11T07:12:27-04:00,Desktop,5,review,"Best airline in the world, seats, food, servic...",Best Airline in the World,0
4,2024-03-10T05:34:18-04:00,Desktop,2,review,Premium Economy Seating on Singapore Airlines ...,Premium Economy Seating on Singapore Airlines ...,0
...,...,...,...,...,...,...,...
9995,2018-08-06T03:48:21-04:00,Desktop,5,review,First part done with Singapore Airlines - acce...,"Flew to NZ 1st half Singapore Airlines, 2nd ha...",1
9996,2018-08-05T22:50:29-04:00,Mobile,5,review,And again a great Flight with Singapore Air. G...,Best Airline,1
9997,2018-08-05T22:47:06-04:00,Desktop,5,review,"We flew business class from Frankfurt, via Sin...",Superb service on Singapore Airlines,1
9998,2018-08-05T20:32:03-04:00,Desktop,4,review,"As always, the A380 aircraft was spotlessly pr...",A Comfortable Fiight Spoiled by lack of adequa...,2


In [ ]:
class FlightInfo(BaseModel):
  """Information about the flight: origin, destination, and the passenger's experience."""
  origin: Optional[str] = Field(description="The departure point of the flight. Can be an airport, a city, or a country.")
  destination: Optional[str] = Field(description="The flight's destination. Can be an airport, a city, or a country.")
  sentiment: str = Field(description="The sentiment of the review. Describe how the reviewer felt about their flight. Use ONLY the words \"good\", \"bad\", \"ok\", or \"unknown\" in lowercase.")

In [ ]:
sentiment_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert in text analysis. Your task is to aggregate information from flight reviews, strictly adhering to the provided instructions."
        ),

        ("human", "{text}"),
    ]
)

In [ ]:
runnable = sentiment_prompt | giga.with_structured_output(schema=FlightInfo)

In [ ]:
summaries = []

for text in reviews['text'].tolist()[10:20]:
  summary = runnable.invoke({"text": text})
  print(summary)
  summaries.append(summary)

origin='Singapore' destination='unknown' sentiment='bad'
origin='EWR' destination='Singapore' sentiment='bad'
origin='Singapore' destination='unknown' sentiment='bad'
origin='unknown' destination='Sydney' sentiment='bad'
origin='London (Heathrow)' destination='Sydney' sentiment='bad'
origin='SFO-SIN' destination='Singapore' sentiment='bad'
origin='Singapore' destination='unknown' sentiment='bad'
origin='unknown' destination='unknown' sentiment='good'
origin='United Kingdom' destination='Singapore' sentiment='bad'
origin='Singapore' destination='Hanoi, Vietnam' sentiment='bad'


In [ ]:
print(reviews['text'][11], '\n', reviews['rating'][11])

Yesterday ( 6 March ) my flight was canceled by the Singapore airlines . After many calls they said that you bought the ticket through booking.com and call them and they should resolve this issue. and singapore airlines didn't want to talk to me anymore. 
I called booking.com and they tell me to call to Singapore airlines.I explain to them that Singapore airlines does not want to help me because I took the tickets through you. 
This went on for several hours. a few hours later Singapore airlines gave me another flight in another state( I bought ticket from JFK NY State , they give EWR NJ ; ) it will cost by taxi around 300$ from NY to Newark 
no matter how bad it was, I agreed and thought that I would fly the next day. but... when I started registering I noticed a mistake they made. 
They gave me a two flights : 
first from EWR - SQ 21	•	7 March 9:35 AM- 8 MARCH 5:25 PM Singapore airport .
 But my second flight was SQ 728  8 MARCH 
 9:50 AM from Singapore .
 How it is possible ? if the